<a href="https://colab.research.google.com/github/afsara-ben/Decision_tree_and_adaboost_implementation/blob/master/Adaboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from pprint import pprint
import io
from sklearn import preprocessing
from collections import Counter
import random
from google.colab import drive
EPSILON = 0.0000000000001

In [2]:
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Colab Notebooks/Adaboost/Dataset/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#Load data

In [0]:
def load_telco_data():
  col_names = ['client_id','Gender', 'seniority', 'partner', 'dependents', 'tenure', 'phoneservice', 'multiple_lines', 'internet_service', 'online_security', 
             'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'contract', 'paperless_billing', 'payment_method',
             'monthly_charges', 'Total_charges', 'label']

  df = pd.read_csv(root_path + 'Telco.csv',header = 0, names=col_names, skiprows=0, skipinitialspace=True, usecols = lambda column : column in 
    ["Gender", "seniority", "partner", "dependents", "tenure", "phoneservice", "multiple_lines", "internet_service", "online_security", 
                 "online_backup","device_protection", "tech_support", "streaming_tv", "streaming_movies", "contract", "paperless_billing", 
                "payment_method","monthly_charges", "Total_charges", "label"])


  col_names.remove("client_id")
  col_names.remove("label")
  target_col = "label"

  label_encoder = preprocessing.LabelEncoder()
  df[target_col] = label_encoder.fit_transform(df['label'])

  return df,col_names



In [0]:
def load_animal_data():
  col_names=["toothed","breathes","legs","label"]

  df = pd.DataFrame({"toothed":["True","True","True","False","True","True","True","True","True","False"],


  "breathes":["True","True","True","True","True","True","False","True","True","True"],

  "legs":["True","True","False","True","True","True","False","False","True","True"],
  "label":["Mammal","Mammal","Reptile","Mammal","Mammal","Mammal","Reptile","Reptile","Mammal","Reptile"]},
  columns = col_names)

  features = df[["toothed","breathes","legs"]]st
  target = df["label"]

  col_names.remove("label")
  target_col = "label"

  label_encoder = preprocessing.LabelEncoder()
  df[target_col] = label_encoder.fit_transform(df['label'])
  
  return df, col_names

In [0]:
#df, col_names = load_animal_data()
df, col_names = load_telco_data()


In [6]:
df.head()

,Gender,seniority,partner,dependents,tenure,phoneservice,multiple_lines,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,Total_charges,label
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1


In [7]:
len(df)

7043

In [8]:
col_names

['Gender',
 'seniority',
 'partner',
 'dependents',
 'tenure',
 'phoneservice',
 'multiple_lines',
 'internet_service',
 'online_security',
 'online_backup',
 'device_protection',
 'tech_support',
 'streaming_tv',
 'streaming_movies',
 'contract',
 'paperless_billing',
 'payment_method',
 'monthly_charges',
 'Total_charges']

#Helper functions


*   *entropy(target_col)*
*   *infoGain(data, split_attribute_name, target_name ="label")*
        - data
        - split_attribute_name, feature whose infoGain is calculated 
        - target_name ="label"
        - returns infogain
        
*   *entropy_before_split()* 
*   *train_test_split()*



In [0]:
def entropy(target_col):
  elements, counts = np.unique(target_col, return_counts= True)

  #print("Elements {} \n" .format(elements))
  #print("Counts {} \n" .format(counts))
  
  entropy = np.sum(
      [
       
      (-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts))
        for i in range(len(elements))
      ]
  )
  return entropy

In [10]:
entropy_before_split = entropy(df['label'])
print("Initial entropy : {} " .format(entropy_before_split))

Initial entropy : 0.8347419473972113 


In [0]:

def infoGain(data, split_attribute_name, target_name ="label"):
  #calculate the entropy of total dataset
  total_entropy = entropy(data[target_name])
  #print("Attribute : {}\n" . format(split_attribute_name))


  element, counts = np.unique(data[split_attribute_name], return_counts=True)

  #print("Elements {} \n" .format(element))
  #print("Counts {} \n" .format(counts))

  weighted_entropy = np.sum(
      [
       (counts[i]/np.sum(counts)) 
       * entropy(data.where(data[split_attribute_name] == element[i]).dropna()[target_name])
       for i in range(len(element))

      ]
  )


  information_gain = total_entropy - weighted_entropy
  #print("Info gain :{}\n". format(information_gain))

  return information_gain

In [12]:
def fix_missing_values():

  null_val_cols = []
  for col in col_names:
    if df[col].isnull().any() == True:
      null_val_cols.append(col)

  print("Column with null values are : {}".format(null_val_cols))


  for col in null_val_cols:
    if str(df[col].dtype) == 'object': #for categorical values
      df[col].fillna(df[col].mode()[0], inplace = True)
    else: #for continuous values
      df[col].fillna(df[col].mean(), inplace = True)
      #df[col].fillna(method = 'ffill', inplace = True)

fix_missing_values()

Column with null values are : ['Total_charges']


In [13]:
#binarization from scratch
#using midpoint to find threshold

%%time
def binarize_telco_data():
  for col in df[['tenure', 'monthly_charges', 'Total_charges']]:
  
      if str(df[col].dtype) == 'float64' or 'int64':
          
          
          print("Working on : {}" .format(col))
          sorted_arr = np.sort(np.unique(pd.to_numeric(df[col], downcast = 'float')))
          #print(sorted_arr)


          cleanedList = [x for x in sorted_arr if str(x) != 'nan']
          #print("length of sorted array : {} and cleaned array : {}".format(len(sorted_arr),len(cleanedList)))
          mid_point = []
          for i in range(len(cleanedList)-1):
              mid_point.append((cleanedList[i]+cleanedList[i+1])/2)
        
          #print(len(mid_point))
          original = np.array(df[col])

          arr = np.array(original).reshape(-1,1)
          arr = [x for x in arr if str(x) != 'nan']
          #print(arr)


          print("Length of original :{} and without empty :{}" .format(len(original), len(arr)))

          igs = []
          count = 0
          for i in mid_point:
              #print("Midpoint :{}" .format(i))
              new_arr = preprocessing.Binarizer(threshold=i).fit_transform(arr)
              df[col] = new_arr.reshape(1,-1)[0]

              #print("\nPrinting df[col]")
              #print(df[col])
              igs.append(infoGain(df,col))
              count += 1

          #print("igs : {}" .format(igs))
        
          threshold = mid_point[np.argmax(igs)]
          print("Threshold is :{}" .format(threshold))
        
          arr = np.array(original).reshape(-1,1)
          mean = np.mean(arr)
          print("Mean :{}" .format(mean))

          new_arr = preprocessing.Binarizer(threshold=threshold).fit_transform(arr)
          df[col] = new_arr.reshape(1,-1)[0]
          #print(df[col])

binarize_telco_data()

Working on : tenure
Length of original :7043 and without empty :7043
Threshold is :17.5
Mean :32.37114865824223
Working on : monthly_charges
Length of original :7043 and without empty :7043
Threshold is :27.674999237060547
Mean :64.76169246059918
Working on : Total_charges
Length of original :7043 and without empty :7043
Threshold is :347.8999938964844
Mean :2283.3004408418656
CPU times: user 8min 57s, sys: 2.85 s, total: 8min 59s
Wall time: 9min 1s


In [14]:
def train_test_split(df):
  percent = int(0.8 * len(df))
  training_data = df.iloc[ : percent].reset_index(drop=True)
  testing_data = df.iloc[ percent : ].reset_index(drop=True)

  return training_data, testing_data

training_data = train_test_split(df)[0]
testing_data = train_test_split(df)[1]

print(len(training_data))
print(len(testing_data))

5634
1409


#Decision tree algorithm

Algo given in assignment - > 

```
function Decision_Tree_Learning(examples, attributes, parent_examples) returns a tree

if examples is empty then return PLURALITY_VALUE(parent_examples)
else if all examples have same classification then return the classification
else if attributes is empty then return PLURALITY_VALUE(examples)
else

  A ← Importance(a, examples)
  tree ← a new decision tree with root test A
  for each value v of A do
    exs ← { e : e ∈ examples and e. A = v }
    subtree ← Decision_Tree_Learning(exs, attributes − A, examples)
    add a branch to tree with label (A = v ) and subtree subtree
  return tree
```
Methods : 

*   *PLURALITY_VALUE* returns the maximum occuring element from a column (mode)
*   *Decision_Tree_Learning*  (

     * *dataset*, input training data, in case of decision stump first and third parameters are same everytime
     * *features*, input features on which training will be done
     * *original_dataset*, input original training data
     * *max_depth*, fix max depth of tree
     * *target_col*, column on which prediction will be done later

    )
    * returns a tree, of type dict

* *find_depth_of_tree* returns depth of tree, which is a dict







In [0]:
def Decision_tree_learning(dataset, features, original_dataset,max_depth, target_col = "label"):

  # --- stopping criteria
  # 1. empty dataset
  if len(dataset) == 0:
    return PLURALITY_VALUE(original_dataset, target_col)

  #2. all rows have same target col
  elif len(np.unique(dataset[target_col])) <=1:
    return np.unique(dataset[target_col])[0]

  #3. attribute of feature is empty
  elif len(features) == 0:
    return PLURALITY_VALUE(dataset, target_col)

  #4. depth limit reached
  elif max_depth == 0: #ki return korbo?
    return PLURALITY_VALUE(original_dataset, target_col)

  #5. grow the tree
  else :
      
      
      best_feature = random.choice(features)

      features = [i for i in features if i != best_feature]

      tree = {best_feature:{}}
      
      
      for v in np.unique(dataset[best_feature]):
       
        sub_data = dataset.where(dataset[best_feature] == v ).dropna()
        
        sub_tree = Decision_tree_learning(sub_data,features,original_dataset,max_depth -1, target_col)
        tree[best_feature][v] = sub_tree

      # ------ the following is in case of normal Decision Tree -----
      # A = [
      #     infoGain(dataset, feature, target_col)
      #     for feature in features
      # ]
      

      # best_feature_index = np.argmax(A)
      # best_feature = features[best_feature_index]

      # features = [i for i in features if i != best_feature]

      # tree = {best_feature:{}}
      
      
      # for v in np.unique(dataset[best_feature]):
      #   sub_data = dataset.where(dataset[best_feature] == v ).dropna()
        
      #   sub_tree = Decision_tree_learning(sub_data,features,original_dataset,max_depth -1, target_col)
      #PLURALITY_VALUE   tree[best_feature][v] = sub_tree
       
  

  return tree

In [0]:
def PLURALITY_VALUE(data, target_col):
  
  elements = np.unique(data[target_col])
  counts = []
  my_list = data[target_col].to_list()
  
  for i in range(len(elements)):
    counts.append(my_list.count(elements[i]))

  max_index = counts.index(max(counts))
  value = elements[max_index]

  return value


In [0]:
def find_depth_of_tree(tree):
  str_tree = str(tree)
  c = 0
  for i in str_tree:
    if i == "{":
       c += 1
  return c-1

In [18]:
#testing Decision_tree_learning()
#PLURALITY_VALUE(df,"label")
Decision_tree_learning(df,col_names, df,1)


{'contract': {'Month-to-month': 0, 'One year': 0, 'Two year': 0}}

#Adaboost implementation



```
function AdaBoost(examples, L_weak, K) returns a weighted majority hypothesis
inputs: examples, set of N labeled examples (x1 , y1 ), ... , (x_n , y_n )
        L_weak, a learning algorithm
        K, the number of hypotheses in the ensemble

local variables: w, a vector of N example weights, initially 1/N
                 h, a vector of K hypotheses
                 z, a vector of K hypothesis weights

for k = 1 to K do
  data ← Resample(examples, w)
  h[k] ← L_weak(data)
  error ← 0
  for j = 1 to N do
    if h[k](x_j) ≠ y_j then error ← error + w[j]
  if error > .5 then continue
  for j = 1 to N do
    if h[k](x_j) = y_j then w[j] ← w[j] * error ⁄ (1 − error)
  w ← Normalize(w)

  Z[k] ← log[(1-error)/error]

return Weighted_Majority(h, z)
```
Note : 
*   decision stumps (h) calculate hobe resampled data er upore
*   weight calculate hobe original data er upore

Methods:

* *predict (query, tree)*
        * query - one row of dataset
        * tree, of dict type
        * returns prediction from the tree

* *normalize* 
    
      * input array to be normalized
      * returns a normalized array whose sum of elements == 1

* *Adaboost (original_data K, features, target_col = "label")*
      * original_data 
      * K = # of stumps
      * features 
      * target_col = "label"
      * returns h,z (array of stumps, sampled weights of stumps)

* *adaboostPredict(data,h,z)*
      * data, testing data
      * h , array of stumps
      * z , array of sampled weights of stumps
      * returns array of prediction

* *adaboostAccuracy (pred,data,target_col="label")*
      * pred , array of prediction
      * data ,testing data
      * target_col="label"
      * returns precision, recall, f1_score, accuracy






In [0]:
def predict(query, tree):
  
  for key in query.keys():   
    if key in list(tree.keys())[0]:
      pred = tree[key][query[key]]
      
  return pred

In [0]:
def normalize(arr):

  total = sum(arr)+ EPSILON
  for i in range(len(arr)):
     arr[i] = (float(arr[i])/float(total))
      

  if(round(sum(arr),10) != float(1)): print("Error in normalizing")
  
  return arr

In [0]:
def Adaboost(original_data, K, features, target_col = "label"):
  w = []
  h = []
  z = []

  size= len(original_data) #original_data.shape[0]
  w = [1/size for i in range(size)]
  
 
  for k in range(0,K):
    data = original_data.copy()  #data = copy of original dataframe
    sampled_data = data.sample(frac = 1,weights = w, replace = True)
    #print(sampled_data)
    h.append(Decision_tree_learning(data, features, original_data,1))
    error = EPSILON

    
    for j in range(size):

      query = dict(original_data.iloc[j]) 
      query = {i:query[i] for i in query if i != target_col} #dropping the target col from dict
      if predict(query,h[k]) != original_data.iloc[j][target_col]:
        error += w[j]
      
      if error > 0.5: continue

    if error == 1: print("Error at :{}" .format(k))
    if error == 0: print("Error = 0 at :{}" .format(k))
    
    for j in range(size):

      query = dict(original_data.iloc[j]) 
      query = {i:query[i] for i in query if i != target_col} #dropping the target col from dict
      if predict(query,h[k]) != original_data.iloc[j][target_col]:
        w[j] = w[j] * (error/(1-error))

      
    
    w = normalize(w)
    

    z.append( np.log((1-error)/error)  )
  print("\nprinting h : {}" .format(h))
  print("printing z :{}\n" .format(z))
  
  return h,z


In [0]:
def adaboostPredict(data,h,z):
  
  pred_final = []
  for i in range(len(data)): #for each row
      pred_label = []
      query =  dict(data.iloc[i, :-1] )
      j = 0
      for stump in h: #for each decision tree
        
        pred = predict(query, stump)
        if(pred == 0): pred = -1 * z[j]
        else: pred = pred * z[j]
        
  
        pred_label.append(pred)
        j+=1
      
    
      pred_final.append(sum(pred_label))

  print(pred_final)
  return pred_final
 

In [0]:
def adaboostAccuracy(pred,data,target_col="label"):
  
  # name = {predicted : actual}
  # --- tp = {Yes : Yes}
  # --- tn = {No : No}
  # --- fp = {Yes : No}
  # --- fn = {No : Yes}
  
  counter, tp,tn,fp,fn = 0,0,0,0,0
  len_ = len(data)
  for i in range(len_):
    
    #print("actual :{}\npred :{}" .format(data[target_col].iloc[i],pred[i]))
    if pred[i] >= 1 and data[target_col].iloc[i] == 1:
      counter +=1
      tp += 1
    elif pred[i] < 1 and data[target_col].iloc[i] == 0:
      counter +=1
      tn  += 1
    elif pred[i] >=1  and data[target_col].iloc[i] == 0:
      fn += 1
    elif pred[i] < 1 and data[target_col].iloc[i] == 1:
      fp += 1

  #print(counter)

  if tp == 0: tp = EPSILON
  if tn == 0: tn = EPSILON
  if fp == 0: fp = EPSILON
  if fn == 0: fn = EPSILON
   
  precision = (tp/(tp+fp)) * 100
  recall = (tp/(tp+fn)) *100
  f1_score = 2 * (precision * recall)/(precision+recall) 
  accuracy = ((tp+tn)/(tp+tn+fp+fn)) * 100

  print("Precision :{}\nRecall :{}\nf1_score :{}\nAccuracy :{}\n" .format(round(precision,5), round(recall,5), round(f1_score,5), round(accuracy,5)))
  #print("Accuracy :{}%" .format((counter/len_)*100))
  
  return round(precision,5), round(recall,5), round(f1_score,5), round(accuracy,5)



In [0]:
#testing function predict()
#tree = {'legs': {'False': 'Mammal', 'True': 'Mammal'}}
#tree = {'streaming_tv': {'No': 0, 'No internet service': 0, 'Yes': 0}}

#query = {'toothed' :'True','breathes' :'True', 'legs':'True' }
#query = {'Gender': 'Female', 'seniority': 0, 'partner': 'No', 'dependents': 'No', 'tenure': 1, 'phoneservice': 'Yes', 'multiple_lines': 'No', 'internet_service': 'No', 'online_security': 'No internet service', 'online_backup': 'No internet service', 'device_protection': 'No internet service', 'tech_support': 'No internet service', 'streaming_tv': 'No internet service', 'streaming_movies': 'No internet service', 'contract': 'One year', 'paperless_billing': 'No', 'payment_method': 'Mailed check', 'monthly_charges': 0.0, 'Total_charges': 1.0}

#predict(query, tree)

In [0]:
#testing function normalize()
#w = [0.16641331257605005, 0.16641331257605005, 0.0003800311359249882, 0.16641331257605005, 0.16641331257605005, 0.16641331257605005, 0.0003800311359249882, 0.0003800311359249882, 0.16641331257605005, 0.0003800311359249882]

#normalize(w)

#Driver functions

In [0]:
#after loading and preprocessing data 
def main():
  booster = [5,10,15,20]

  precision_, recall_, f1_score_, accuracy_ = [],[],[],[]
  #for i in range(10): #run adaboost 10 times
  for j in booster:
    
      print("\nRunning Adaboost for  k = {}\n" .format(j))
      h,z = Adaboost(df,j, col_names)

      pred_ = adaboostPredict(testing_data, h,z)
      pres,rcall,f1,acc = adaboostAccuracy(pred_, testing_data)
    
      precision_.append(pres)
      recall_.append(rcall)
      f1_score_.append(f1)
      accuracy_.append(acc)

  print("Precision :{}\nRecall :{}\nf1_score :{}\nAccuracy :{}\n" .format(np.unique(precision_), np.unique(recall_),np.unique(f1_score_),np.unique(accuracy_)))
  

In [27]:
from time import time 

if __name__ == "__main__":
    start_time = time()
    main()
    time_taken = (time() - start_time)/60.0
    print('Runtime: {} minutes' .format(time_taken))


Running Adaboost for  k = 5


printing h : [{'internet_service': {'DSL': 0, 'Fiber optic': 0, 'No': 0}}, {'device_protection': {'No': 0, 'No internet service': 0, 'Yes': 0}}, {'online_backup': {'No': 0, 'No internet service': 0, 'Yes': 0}}, {'monthly_charges': {0.0: 0, 1.0: 0}}, {'payment_method': {'Bank transfer (automatic)': 0, 'Credit card (automatic)': 0, 'Electronic check': 0, 'Mailed check': 0}}]
printing z :[1.0182425552899539, 2.036485110579627, 4.072970221154041, 8.145940441968916, 16.291879694483644]

[-31.56551802347618, -31.56551802347618, -31.56551802347618, -31.56551802347618, -31.56551802347618, -31.56551802347618, -31.56551802347618, -31.56551802347618, -31.56551802347618, -31.56551802347618, -31.56551802347618, -31.56551802347618, -31.56551802347618, -31.56551802347618, -31.56551802347618, -31.56551802347618, -31.56551802347618, -31.56551802347618, -31.56551802347618, -31.56551802347618, -31.56551802347618, -31.56551802347618, -31.56551802347618, -31.56551802347618, -